In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
class NutritionFacts(BaseModel):
  calories: float = Field(description='Total calories')
  fat: float = Field(description='Total fat in grams')
  carbohydrates: float = Field(description='Total carbohydrates in grams')
  sodium: float = Field(description='Total sodium in milligrams')
parser = JsonOutputParser(pydantic_object=NutritionFacts)

In [3]:
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"calories": {"title": "Calories", "description": "Total calories", "type": "number"}, "fat": {"title": "Fat", "description": "Total fat in grams", "type": "number"}, "carbohydrates": {"title": "Carbohydrates", "description": "Total carbohydrates in grams", "type": "number"}, "sodium": {"title": "Sodium", "description": "Total sodium in milligrams", "type": "number"}}, "required": ["calories", "fat", "carbohydrates", "sodium"]}\n```'

In [4]:
from langchain_community.llms.databricks import Databricks
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

In [5]:
import os
os.environ.get('DATABRICKS_HOST')

'https://e2-demo-west.cloud.databricks.com'

In [6]:
llm = Databricks(endpoint_name='databricks-mpt-30b-instruct', extra_params={'temperature': 0.0})

In [7]:
prompt = PromptTemplate.from_template(template="""What are the nutrition facts for a single {fruit}? You must format your output as a JSON object.\n{format_instructions}""",
                                     partial_variables={'format_instructions': format_instructions})
prompt

PromptTemplate(input_variables=['fruit'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"calories": {"title": "Calories", "description": "Total calories", "type": "number"}, "fat": {"title": "Fat", "description": "Total fat in grams", "type": "number"}, "carbohydrates": {"title": "Carbohydrates", "description": "Total carbohydrates in grams", "type": "number"}, "sodium": {"title": "Sodium", "description": "Total sodium in milligrams", "type": "number"}}, "required": ["calories", "fat", "carbohydrates", "sodium"]}\n```'}, te

In [8]:
# The standard output_parser doesn't work with MPT-30b due to extra characters
# Hence we are defining our own "extract_json" function
from extract_json import extract_json

In [9]:
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)  # output_parser=parser, 
#chain = prompt | llm
chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['fruit'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"calories": {"title": "Calories", "description": "Total calories", "type": "number"}, "fat": {"title": "Fat", "description": "Total fat in grams", "type": "number"}, "carbohydrates": {"title": "Carbohydrates", "description": "Total carbohydrates in grams", "type": "number"}, "sodium": {"title": "Sodium", "description": "Total sodium in milligrams", "type": "number"}}, "required": ["calories", "fat", "carbohy

In [10]:
avocado_nutrition_facts = extract_json(chain.run('avocado'))
avocado_nutrition_facts

/Users/vijay.balasubramaniam/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
What are the nutrition facts for a single avocado? You must format your output as a JSON object.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"calories": {"title": "Calories", "description": "Total calories", "type": "number"}, "fat": {"title": "Fat", "description": "Total fat in grams", "type": "number"}, "carbohydrates": {"title": "Carbohydrates", "description": "Total carbohydrates in grams", "type": "number"}, "sodium": {"title": "Sodium", "description": "Total sodium in milligrams", "type": "number"}}, "requ

{'calories': 255, 'fat': 21.0, 'carbohydrates': 14.0, 'sodium': 1.0}

In [11]:
apple_nutrition_facts = extract_json(chain.run('apple'))
apple_nutrition_facts



> Entering new LLMChain chain...
Prompt after formatting:
What are the nutrition facts for a single apple? You must format your output as a JSON object.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"calories": {"title": "Calories", "description": "Total calories", "type": "number"}, "fat": {"title": "Fat", "description": "Total fat in grams", "type": "number"}, "carbohydrates": {"title": "Carbohydrates", "description": "Total carbohydrates in grams", "type": "number"}, "sodium": {"title": "Sodium", "description": "Total sodium in milligrams", "type": "number"}}, "requir

{'calories': 52, 'fat': 0.2, 'carbohydrates': 13.5, 'sodium': 0}

In [12]:
avocado_nutrition_facts, apple_nutrition_facts

({'calories': 255, 'fat': 21.0, 'carbohydrates': 14.0, 'sodium': 1.0},
 {'calories': 52, 'fat': 0.2, 'carbohydrates': 13.5, 'sodium': 0})